# CNN for Cifar-10 Dataset

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

### Requirements

For this project we are used:
* Python 3.6
* Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1
* or CPU optimized tensorflow for intel

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_io, graph_util
from tensorflow.tools.graph_transforms import TransformGraph

from cnn.dense import (NET_NAME, dataset_preprocessing_by_keras, eval_fn,
                       forward_pass, loss_fn)
from cnn.model_class import MODELS_DIR, TfClassifier
from cnn.utils.dataset import load_cifar10
from cnn.utils.save_models import load_frozen_graph

## Cifar-10 Dataset

Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [2]:
x_train, t_train, x_test, t_test = load_cifar10()

In [3]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

It also needs to be normalized

In [4]:
x_train = dataset_preprocessing_by_keras(x_train)
x_train[0, :, :, 0]

array([[-0.68747891, -0.65572952, -0.63985482, ..., -0.95734874,
        -1.00497283, -0.92559935],
       [-0.65572952, -0.63985482, -0.59223073, ..., -0.92559935,
        -1.02084753, -0.86210057],
       [-0.7192283 , -0.63985482, -0.65572952, ..., -0.68747891,
        -0.735103  , -0.70335361],
       ...,
       [-0.36998499, -0.7509777 , -1.00497283, ..., -0.25886212,
        -0.24298742, -0.14773924],
       [-0.25886212, -0.65572952, -1.19546919, ...,  0.31262694,
        -0.00486698, -0.49698256],
       [-0.27473681, -0.35411029, -1.11609571, ...,  0.75711843,
         0.64599556,  0.28087755]])

## CNN Model and Training

We will use a custom made wrapper for tensorfllow NN training and use

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [5]:
model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

This network is trained for 50 epochs.

In [7]:
history = model.fit(
    [x_train, t_train],
    batch_size=64,
    validation_split=0.1,
    epochs=1,
    verbosity=1,
    keep_prob=0.5)

print(history)

For training: tensorboard --logdir=/tmp/log-tb/dense_cnn/training
For validation: tensorboard --logdir=/tmp/log-tb/dense_cnn/validation
{'logits': (array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),), 'classes': (array([0, 0, 0, ..., 0, 0, 0]),), 'probabilities': (array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32),), 'accuracy': ((0.102, 0.1002),), 'mse': ((nan, nan),), 'loss': (nan,), 'summaries': (b'\n\x0b\n\x04loss\x15\xff\xff\xff\xff\n\x11\n\naccuracy_1\x15`\xe5\xd0=\n\n\n\x03mse\x15\x00\x00\xc0\xff',)}


Then it's evaluated

In [8]:
evals = model.evaluate([x_test, t_test])

print(evals)

INFO:tensorflow:Restoring parameters from /home/daibak/Documents/Code/Python/aca-tensorflow/cnn/models/dense_cnn/model.ckpt
[{'logits': array([[  58.449165 ,  -40.94816  ,   -5.6226172, ...,  -94.2457   ,
          43.86064  ,  -68.44922  ],
       [  -9.421125 ,   -2.6976483,  -56.005733 , ...,  -15.75792  ,
          16.243078 ,  -80.76974  ],
       [  88.704    ,  -35.250233 ,  -27.641563 , ..., -143.42578  ,
         217.6601   ,   21.607813 ],
       ...,
       [ -35.267845 ,   -7.7263284,  -25.900536 , ...,   25.24029  ,
           5.2269926,  -20.619165 ],
       [  45.99917  ,  119.776566 , -125.03786  , ...,  -61.233307 ,
         179.8653   ,   47.13309  ],
       [  22.152536 ,  -10.786198 ,  -36.33955  , ..., -121.99178  ,
         116.80828  ,   32.390125 ]], dtype=float32), 'classes': array([0, 3, 8, ..., 5, 8, 8]), 'probabilities': array([[9.9999952e-01, 0.0000000e+00, 1.4927161e-28, ..., 0.0000000e+00,
        4.6161975e-07, 0.0000000e+00],
       [3.5268134e-31, 2.93

## Optimization Transform

In [18]:
model.freeze_graph()
graph, inp, out = model.load_model()

In [19]:
opt_graph = model.optimize_for_inference(add_transf=["sort_by_execution_order"])
opt_graph.get_operations()

[<tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'conv2d/kernel' type=VariableV2>,
 <tf.Operation 'conv2d/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d/bias' type=VariableV2>,
 <tf.Operation 'conv2d/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d/MaxPool' type=MaxPool>,
 <tf.Operation 'conv2d_1/kernel' type=VariableV2>,
 <tf.Operation 'conv2d_1/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d_1/bias' type=VariableV2>,
 <tf.Operation 'conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d_1/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d_1/MaxPool' type=MaxPool>,
 <tf.Operation 'flatten/Shape' type=Shape>,
 <tf.Operation 'flatten/strided_slice/stack' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_1' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_2' type=Const>,
 <tf.Operation 'flatten/strided_slice' type=StridedSlice>,
 <tf.Operation 'flatten/Reshape/shape/1' type=Const>,
 <tf.Operation 'flatten/Reshape/shape' typ

In [20]:
graph.get_operations()

[<tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'conv2d/kernel' type=Const>,
 <tf.Operation 'conv2d/kernel/read' type=Identity>,
 <tf.Operation 'conv2d/bias' type=Const>,
 <tf.Operation 'conv2d/bias/read' type=Identity>,
 <tf.Operation 'conv2d/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d/MaxPool' type=MaxPool>,
 <tf.Operation 'conv2d_1/kernel' type=Const>,
 <tf.Operation 'conv2d_1/kernel/read' type=Identity>,
 <tf.Operation 'conv2d_1/bias' type=Const>,
 <tf.Operation 'conv2d_1/bias/read' type=Identity>,
 <tf.Operation 'conv2d_1/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d_1/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d_1/MaxPool' type=MaxPool>,
 <tf.Operation 'flatten/Shape' type=Shape>,
 <tf.Operation 'flatten/strided_slice/stack' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_1' type=Const>,
 <tf.Operation 'flatten/str